<a href="https://colab.research.google.com/github/jasonwillschiu/scratch/blob/master/meetup%20bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# selenium and chromium install
!pip install selenium
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  chromium-browser chromium-browser-l10n chromium-codecs-ffmpeg-extra
Suggested packages:
  webaccounts-chromium-extension unity-chromium-extension adobe-flashplugin
The following NEW packages will be installed:
  chromium-browser chromium-browser-l10n chromium-chromedriver
  chromium-codecs-ffmpeg-extra
0 upgraded, 4 newly installed, 0 to remove and 11 not upgraded.
Need to get 79.2 MB of archives.
After this operation, 268 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 chromium-codecs-ffmpeg-extra amd64 85.0.4183.83-0ubuntu0.18.04.2 [1,118 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 chromium-browser amd64 85.0.4183.83-0ubuntu0.18.04.2 [70.3 MB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 chromium-browser-l10n all 85.0.418

In [7]:
# authenticate for colab to look at gcs
from google.colab import auth
auth.authenticate_user()
print("auth'ed for colab")

In [8]:
import pickle
import selenium.webdriver
# use tf to read files from gcs
import tensorflow as tf 
from tensorflow.python.lib.io import file_io

# the group to look inside
TEST_URL   = 'https://www.meetup.com/en-AU/Fishburners-Meetup/events/'
TOP_SHOT_EVENTS_URL = 'https://www.meetup.com/Topshot-Badminton-Meetup/events/'

# cookie location local
#cookie_name = 'meetup_cookies_chrome_2020-09-19.pkl'

# cookie location on gcs
cookie_gcs = 'gs://meetup-script/meetup_cookies_chrome_2020-09-19.pkl'

# picking the chrome driver
OPTIONS = webdriver.ChromeOptions()
# remove all these when watching it the first time
OPTIONS.add_argument('-headless') # no browser
OPTIONS.add_argument('-no-sandbox') # not a test browser
OPTIONS.add_argument('-disable-dev-shm-usage') # so docker will allocate >64MB to chrome
# if we need to be a realer browser (not a headless chrome)
OPTIONS.add_argument('--user-agent=""Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.157 Safari/537.36""')
driver = webdriver.Chrome('chromedriver',options=OPTIONS)
print('chromedriver - loaded')

# load the page without the cookie
driver.get('https://meetup.com')
print('meetup.com - loaded without cookie')
# load the cookie
with file_io.FileIO(cookie_gcs,'rb') as f:
  cookies = pickle.load(f)    
for cookie in cookies:
    driver.add_cookie(cookie)
# load the page again after the cookie is loaded, this is the same as logging in
driver.get('https://meetup.com')
print('meetup.com - loaded with cookie')

# go to our meetup group
driver.get(TEST_URL)
print('meetup group URL - loaded')

# find events to attend, this will be in a funky format
elem = driver.find_elements_by_partial_link_text("Attend")
# grab the actual URLs and remove the ?action bit
elems = []
for e in elem[:4]:
    # remove the action rsvp yes part, because that doesn't seem to work (19 Sep 2020)
    elems.append(e.get_attribute('href').split('?action')[0])
    
# go to the first event page
driver.get(elems[0])
print('meetup group event - URL loaded')

# click attend
driver.find_element_by_class_name('gtmEventFooter--attend-btn').click()
print('meetup group event - clicked attend')

chromedriver - loaded
meetup.com - loaded without cookie
meetup.com - loaded with cookie
meetup group URL - loaded
meetup group event - URL loaded
meetup group event - clicked attend
